In [ ]:
!apt install ../input/pyturbojpeg/libturbojpeg_1.4.2-0ubuntu3.4_amd64.deb
!pip install ../input/pyturbojpeg/PyTurboJPEG-1.4.1

In [ ]:
%%time
!pip install ../input/wheat-detection/yacs-0.1.7-py3-none-any.whl
!pip install ../input/wheat-detection/torch-1.5.0cu101-cp37-cp37m-linux_x86_64.whl
!pip install ../input/wheat-detection/fvcore-0.1.1.post20200716-py3-none-any.whl
!pip install ../input/wheat-detection/pycocotools-2.0.1-cp37-cp37m-linux_x86_64.whl
!pip install ../input/wheat-detection/torchvision-0.6.0cu101-cp37-cp37m-linux_x86_64.whl
!pip install ../input/wheat-detection/detectron2-0.2cu101-cp37-cp37m-linux_x86_64.whl

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math 

import torch
import torch.nn as nn

import cv2

from tqdm.notebook import tqdm # Progress Bar 

import os
import sys
sys.path.append("../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master")


In [ ]:
label_dic = {
    0: 'healthy', 
    1: 'scab',
    2: 'rust',
    3: 'frog_eye_leaf_spot',
    4: 'complex', 
    5: 'powdery_mildew'
}

num_classes = 6

device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

In [ ]:
from turbojpeg import TurboJPEG

jpeg_reader = TurboJPEG()

def read_img(img):
    with open(img, "rb") as f:
        return jpeg_reader.decode(f.read(), 0) 

In [ ]:
def padding_img(img, inp_dim):
    '''resize image with unchanged aspect ratio using padding'''
      
    img_w, img_h = img.shape[1], img.shape[0]
    w, h = inp_dim
    new_w = int(img_w * min(w/img_w, h/img_h))
    new_h = int(img_h * min(w/img_w, h/img_h))

    resized_image = cv2.resize(img, (new_w,new_h), interpolation = cv2.INTER_AREA)#cv2.INTER_CUBIC

    #create a black canvas    
    canvas = np.full((inp_dim[1], inp_dim[0], 3), 0, dtype=np.uint8)
 
    #paste the image on the canvas
    canvas[(h-new_h)//2:(h-new_h)//2 + new_h,(w-new_w)//2:(w-new_w)//2 + new_w,  :] = resized_image
    
    
    return canvas

In [ ]:
import detectron2
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.logger import setup_logger

setup_logger()


def get_predictor():
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.DATASETS.TRAIN = ()
    cfg.DATALOADER.NUM_WORKERS = 16

    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = (8)  # faster, and good enough for this toy dataset
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  # 3 classes (data, fig, hazelnut)

    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
    
    cfg.MODEL.WEIGHTS = "../input/image-segmentation-using-detectron2/model.pth"
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
    predictor = DefaultPredictor(cfg)
    return predictor

def get_cropped_leaf(img,predictor): 
    #get prediction
    outputs = predictor(img)
    
    #get boxes and masks
    ins = outputs["instances"]
    pred_masks = ins.get_fields()["pred_masks"]
    boxes = ins.get_fields()["pred_boxes"]    
    
    #get main leaf mask if the area is >= the mean area of boxes and is closes to the centre 
    masker = pred_masks[np.argmin([calculateDistance(x[0], x[1], int(img.shape[1]/2), int(img.shape[0]/2)) for i,x in enumerate(boxes.get_centers()) if (boxes[i].area()>=torch.mean(boxes.area()).to("cpu")).item()])].to("cpu").numpy().astype(np.uint8)

    #mask image
    mask_out = cv2.bitwise_and(img, img, mask=masker)
    
    #find contours and boxes
    contours, hierarchy = cv2.findContours(masker.copy() ,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contour = contours[np.argmax([cv2.contourArea(x) for x in contours])]
    rotrect = cv2.minAreaRect(contour)
    box = cv2.boxPoints(rotrect)
    box = np.int0(box)
    
    #crop image
    resized = get_cropped(rotrect,box,mask_out)
    
    return resized

#function to crop the image to boxand rotate

def get_cropped(rotrect,box,image):
    
    width = int(rotrect[1][0])
    height = int(rotrect[1][1])

    src_pts = box.astype("float32")
    # corrdinate of the points in box points after the rectangle has been
    # straightened
    dst_pts = np.array([[0, height-1],
                        [0, 0],
                        [width-1, 0],
                        [width-1, height-1]], dtype="float32")

    # the perspective transformation matrix
    M = cv2.getPerspectiveTransform(src_pts, dst_pts)

    # directly warp the rotated rectangle to get the straightened rectangle
    warped = cv2.warpPerspective(image, M, (width, height))
    return warped

def calculateDistance(x1,y1,x2,y2):  
    dist = math.hypot(x2 - x1, y2 - y1)
    return dist 



In [ ]:
predictor = get_predictor()

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensor

def transform_valid():
    
    augmentation_pipeline = A.Compose(
        [
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
                ),
            ToTensor() 
        ],
        p = 1
    )
    return lambda img:augmentation_pipeline(image=np.array(img))['image']


def transform_vvv():
    
    augmentation_pipeline = A.Compose(
        [
            A.SmallestMaxSize(224),
            A.RandomCrop(224, 224),
        ],
        p = 1
    )
    return lambda img:augmentation_pipeline(image=np.array(img))['image']


In [ ]:
from efficientnet_pytorch import model as enet

model = enet.EfficientNet.from_name('efficientnet-b4')
model._fc = nn.Linear(in_features=model._fc.in_features, out_features=num_classes)
model.load_state_dict(torch.load('../input/tmpth/best_model(4).pth'))

model = model.to(device)

In [ ]:
from glob import glob
valid_image_list = glob('../input/plant-pathology-2021-fgvc8/test_images/*.jpg')

In [ ]:

def to_lab(preds):
    return ((preds > 0) + torch.nn.functional.one_hot(preds.argmax(1), 6) != 0).long()

model.eval()
predict_list = []
image_name_list = []

for i, image in tqdm(enumerate(valid_image_list)) :
    image_name = image[48:]
    
    
    img = read_img(image)
    img = padding_img(get_cropped_leaf(img, predictor),(224,224))
    img = transform_valid()(img)
    
    img = img.to(device)
    img = img.reshape(-1, 3, 224, 224)
    
    with torch.set_grad_enabled(False):
        predict = model(img)

    predict_list.append(list(to_lab(predict).reshape(-1).nonzero(as_tuple=False).reshape(-1).cpu().numpy()))
    image_name_list.append(image_name)
    

In [ ]:
#predict_list = np.array(predict_list)
image_name_list = np.array(image_name_list)

submission_df = pd.DataFrame()
submission_df['image'] = image_name_list
submission_df['label_id'] = predict_list
submission_df['labels'] = submission_df['label_id'].apply(lambda x: " ".join([label_dic[i] for i in x]))
del submission_df['label_id']

In [ ]:
submission_df.to_csv("submission.csv", index = False)